<a href="https://colab.research.google.com/github/backeung/KoBERT-multi-classification-model/blob/main/KoBERT_A%2BB%EC%82%AC%EC%A0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   KoBERT 다중분류모델(기존): https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab
*   기존 KoBERT 코드: https://github.com/Seolini/KoBERT_Korean_multi_classification/blob/main/KoBERT_%ED%95%9C%EA%B5%AD%EC%96%B4_7%EA%B0%9C%EA%B0%90%EC%A0%95_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98.ipynb
*   ver2: https://github.com/SKTBrain/KoBERT/issues/67#L55


*   SKT Korean BERT: https://reposhub.com/python/deep-learning/SKTBrain-KoBERT.html

*   get_kobert_model(): https://github.com/monologg/KoBERT-Transformers
* 코드수정1: https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf
* 코드수정2: https://github.com/SKTBrain/KoBERT/issues/68

**1. Colab 환경설정**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/Shareddrives/종합설계2/Colab Notebooks/KoBERT'

/content/drive/Shareddrives/종합설계2/Colab Notebooks/KoBERT


In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

fatal: destination path 'KoBERT' already exists and is not an empty directory.


In [ ]:
cd 'KoBERT'

/content/drive/Shareddrives/종합설계2/Colab Notebooks/KoBERT/KoBERT


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 46.9 MB 1.3 MB/s 
     |████████████████████████████████| 344 kB 56.8 MB/s 
     |████████████████████████████████| 1.2 MB 66.6 MB/s 
     |████████████████████████████████| 4.8 MB 67.1 MB/s 
     |████████████████████████████████| 1.3 MB 51.5 MB/s 
     |████████████████████████████████| 1.1 MB 45.4 MB/s 
     |████████████████████████████████| 2.9 MB 51.4 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595737 sha256=564314d7d4db8262717739d2e20d8ca98e20e1ec6a1f8a02da5b26e3acbb4a71
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
!pip install .

Processing /content/drive/Shareddrives/종합설계2/Colab Notebooks/KoBERT/KoBERT
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12780 sha256=980333b978e9b65c4983517c124de30d4bb66839d3b1f1d6cf76e5e1fc88e1aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-t7nhqv7j/wheels/1a/58/1f/6cfd2b2e6f63f207a90ec7b093d471d28a4f33de7c0f3af5fb
Successfully built kobert


In [ ]:
predefined_args = {
        'attention_cell': 'multi_head',
        'num_layers': 12,
        'units': 768,
        'hidden_size': 3072,
        'max_length': 512,
        'num_heads': 12,
        'scaled': True,
        'dropout': 0.1,
        'use_residual': True,
        'embed_size': 768,
        'embed_dropout': 0.1,
        'token_type_vocab_size': 2,
        'word_embed': None,
    }

In [ ]:
!pip install torch==1.7.0

     |████████████████████████████████| 776.7 MB 4.3 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.0 which is incompatible.


In [ ]:
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#참고: 코드수정1
import torch
from kobert.pytorch_kobert import get_kobert_model, get_pytorch_kobert_model
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-sov0rgb_/kobert-tokenizer_699dddcdb1c44663a2435527a2af0c18
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-sov0rgb_/kobert-tokenizer_699dddcdb1c44663a2435527a2af0c18
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4682 sha256=c4df72000f87d11439a34c9fd222772c0fec26ec0d2558c13a595fd784237c34
  Stored in directory: /tmp/pip-ephem-wheel-cache-1hr8v07v/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

get_kobert


Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

get_kobert


In [ ]:
print(bertmodel, vocab)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )

https://github.com/SKTBrain/KoBERT/issues/68
https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf

**2. 데이터셋전처리**

In [ ]:
#데이터 불러오기
import pandas as pd
chatbot_data = pd.read_csv('/content/drive/Shareddrives/종합설계2/Dataset/DataTest.csv')

In [ ]:
chatbot_data

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
78581,노인정 박 씨가 우울증을 앓고 있었다는데 나는 아무것도 몰랐어. 충격적이야.,놀람,NaN,NaN,NaN,NaN,NaN
78582,안 그래도 일이 많은데 이번에 상사가 실수한 걸 내가 다 수습하게 생겼어.,슬픔,NaN,NaN,NaN,NaN,NaN
78583,내가 다니는 학원 선생님은 학생들을 부모님 직업과 재산으로 구분 지어. 굉장히 속물이야.,혐오,NaN,NaN,NaN,NaN,NaN
78584,나이를 먹으니 사람이 그리워서 자녀들 연락에 안달하게 돼.,슬픔,NaN,NaN,NaN,NaN,NaN


In [ ]:
chatbot_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77188 entries, 0 to 77187
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  77187 non-null  object
 1   Emotion   77188 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [ ]:
#NaN값을 가지는 컬럼들을 없애고 Emotion컬럼에 7가지의 감정 단어들을 숫자로 변경
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [ ]:
#data_list = ('문장', '감정')
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(type(data_list))

<class 'list'>


In [ ]:
data_list

[['언니 동생으로 부르는게 맞는 일인가요..??', '0'],
 ['그냥 내 느낌일뿐겠지?', '0'],
 ['아직너무초기라서 그런거죠?', '0'],
 ['유치원버스 사고 낫다던데', '0'],
 ['근데 원래이런거맞나요', '0'],
 [' 남자친구가 떠날까봐요', '0'],
 ['이거 했는데 허리가 아플수도 있나요? ;;', '0'],
 ['내가불안해서꾸는걸까..', '0'],
 [' 일주일도 안 남았당...ㅠㅠ', '0'],
 ['약은 최대한 안먹으려고 하는데좋은 음시있나요?0', '0'],
 [' 구직활동 하면서 남는시간은 뭘로 활용해야 되지..', '0'],
 ['괜찮은분 같아서 괜히 조급해지네요 ㅜ', '0'],
 [' 제가 스타일을 바꾸면 더 어색하게만 변할것같아서 ㅠ0', '0'],
 ['아내 있으면 여자 있는 술집가면 법 적으로 문제있나요?', '0'],
 [' 한소리들을지.... 도와주세요', '0'],
 ['도와주실분 없나요..', '0'],
 ['그냥 잡다한거말고학업에 관련된거 듣는게 더 좋은가요?', '0'],
 [' 가격은 반값....이거 질이 다른건가요??', '0'],
 ['마지막에 말씀 드렸던대로 글은 이제 내릴게요의도치않게 실시간베스트글에 올라가서 남친이 볼거 같아서요 ㅠㅠ', '0'],
 ['화장품회사다보니, 젊은 여자들이 많은거 같은데 걱정이네요..', '0'],
 [' 몸무게 1키로찌는건 아니겠죠?', '0'],
 ['o자 다리 많이 심한가요?', '0'],
 [' 안하는게 나을까요??', '0'],
 ['그정도로 더러운가요.........', '0'],
 [' 보통 가진통도 이렇게 오래가나요?', '0'],
 ['여자가 술취해서 먼저 전화하는거 짜증나요???', '0'],
 ['걱정이 큽니다', '0'],
 ['이거 무슨 증상이죠??ㅜㅜ', '0'],
 [' 다름 아니라 제동생 걱정 입니다', '0'],
 ['아무래도 무리겠죠?', '0'],
 ['애인으로써 정말 걱정됩니다', '0'],
 ['이러다 일주일전에 나오는

**3. Train Data & Test Data**

In [ ]:
#train:test = 7:3
from sklearn.model_selection import train_test_split
                                                         
train,test = train_test_split(data_list, test_size=0.3, random_state=1234)

In [ ]:
print(len(train)) # 54031
print(len(test)) # 23157

55010
23576


In [ ]:
test

[['직장에서 부당한 일을 겪어서 너무 화가 나.', '2'],
 ['일단생각해놓는건 알바인데...ㅠㅜ', '3'],
 ['너때문에 노잼이였다 잘가라', '6'],
 ['열심히 다이어트를 했는데 그대로야.', '3'],
 ['기자놈 반미감정 만들려고 애쓰네 ㅉㅉ', '6'],
 ['전기차 충전 인프라좀 제대로 갖추자', '2'],
 ['박근혜가청와대에서안나오면우리가처들어간다~~~~빠루들고', '6'],
 ['7천만대 판매 했는데 실패라고 하면 역시 무서운 중국', '1'],
 ['오늘 소개팅을 했는데 상대방이 나한테 전화번호를 안 주더라고. 거참 왜지?', '0'],
 ['부모님은 내가 좋아하는 건 관심이 없고 오직 내 성적만 중요하게 생각하셔.', '3'],
 ['혹시 다른 분들도 이러신 적 있으신가요?ㅠㅠ', '0'],
 ['저색기들 다 때려 죽여버리고 싶다', '2'],
 ['마누라의 잔소리에 상처를 입었어. 상처를 고치려면 어떻게 하면 될까?', '0'],
 ['아내와 대화를 나누는 게 잘 안 돼.', '3'],
 ['어제 푸른바다 홍진경보고 빵터짐ㅋ', '5'],
 ['출산해서 아이를 키워야 하는 데 형편이 좋지 않아 걱정이야.', '0'],
 ['새로 산 자동차 앞 좌석 시트에 친한 친구가 커피를 쏟아서 너무 짜증이 나.', '0'],
 ['미래가 없네요 저는..', '3'],
 ['결혼이 코앞인데 내가 옳은 결정을 한 건지 불안해. 식장에 들어가는 게 맞는 건지 감정이 복잡해.', '0'],
 ['아시아 최강국 경제군사 대국 대중화제국 아시아 제3경제대국 선진국 G7대일본제국 제2군사대국 러시아 제국 세계 최강국 미합중국 동북아의 찐따 빵셔틀 개한망국',
  '6'],
 ['요즘 아들과 사이가 안 좋은 것 같아서 괴로워.', '3'],
 ['너무 마음이 상쾌해.', '5'],
 ['유재석 김종민은 연예계성자다', '5'],
 ['서장훈:도대체 이게 무슨 의미가 있습니까~', '4'],
 ['정말 난감하네.', '0'],
 [' 이렇게 논란이 많은 

In [ ]:
train

[['제가 잘못한건가요?', '0'],
 ['노무현 대통령 죽인새끼!!!!', '2'],
 ['형님 축하드립니다!!', '5'],
 ['나를 사랑해준다는 느낌은 많이 드는데 이런거 보면 내가 외모가 별로인가 라는생각도 들고 ㅠㅠ....', '0'],
 ['자수성가한건 대단한데.', '1'],
 ['이야 사람 간 보는것도 아니고 주자 1-3루 만들어놓고 삼진잡네 ㅋㅋ 쪼는맛은 최고다 파팅', '5'],
 ['어젠 정말 형의 모습이 멋져 보이던지~~', '5'],
 [' 그래서 안나오다가 화장실가서 긴장풀고 배에힘풀면 주르륵 나오능겨;;', '1'],
 ['내가 그런 듯 한데.....', '4'],
 ['저는 사귀면서 권태기가 온적이 없어서 잘 모르겠어요', '0'],
 ['거 경질되기 딱 좋은 날씨네', '6'],
 ['스왑딜 해도 모자랄판에 돈까지 더해준다고??ㅋㅋ', '1'],
 ['그래서 2016kbs 연기대상이 송송커플 이라거임?', '1'],
 ['왜 이렇게들 생각할가?', '0'],
 ['내가 구글 폰 계정에 동기화를 안한거 같은데 어쩌죠 ///', '0'],
 ['먼지나게 털어라 검찰 제발', '2'],
 [' 그래서 8년우정이깨지고있는데 불안해요', '0'],
 ['박근혜가 피노키오였다면 코가 얼마나 길어졌을까?', '2'],
 ['아님힘들까요?', '0'],
 ['분명 빠른것이 아닐진데, 또 죄책감이 들고 쓸데없는 고민이 생기네요...', '0'],
 ['오지은 정글의법칙 보고나서 꼴도 보기싫음', '2'],
 [' 너무힘드네요ㅜ', '3'],
 ['광고는 순수익인데.. 매출 8프로면 많은거 아닌가?', '4'],
 ['사진으로만 봤지 동영상으로 보니 정말 신기하다', '1'],
 ['진짜 돈벌기쉽다.ㅈ.', '1'],
 ['의외로 기능도 어려워 하시던데.', '1'],
 ['얘까지 대통령 해먹으면 그땐 끝장이다', '2'],
 ['오해하는 것 같다', '3'],
 [' 끝까지 맞으면서도 상대에게 주먹한번 내밀지 않으신점!!', '1'],
 ['

**4.KoBERT 입력 데이터로 만들기**

In [ ]:
#BERTDataset 클래스 생성
#각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등 실시
#BERTSentenceTransform 이라는 모듈사용-> 토큰화, 패딩 이루어짐
#참고: 코드수정2 ->class에서 def init안에 vocab추가 
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
#하이퍼 파라미터 조정
#사실 파라미터의 값은 정답이 없고 가장 좋은 성능을 내는 값을 찾아가는 것이 좋음
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
test

[['직장에서 부당한 일을 겪어서 너무 화가 나.', '2'],
 ['일단생각해놓는건 알바인데...ㅠㅜ', '3'],
 ['너때문에 노잼이였다 잘가라', '6'],
 ['열심히 다이어트를 했는데 그대로야.', '3'],
 ['기자놈 반미감정 만들려고 애쓰네 ㅉㅉ', '6'],
 ['전기차 충전 인프라좀 제대로 갖추자', '2'],
 ['박근혜가청와대에서안나오면우리가처들어간다~~~~빠루들고', '6'],
 ['7천만대 판매 했는데 실패라고 하면 역시 무서운 중국', '1'],
 ['오늘 소개팅을 했는데 상대방이 나한테 전화번호를 안 주더라고. 거참 왜지?', '0'],
 ['부모님은 내가 좋아하는 건 관심이 없고 오직 내 성적만 중요하게 생각하셔.', '3'],
 ['혹시 다른 분들도 이러신 적 있으신가요?ㅠㅠ', '0'],
 ['저색기들 다 때려 죽여버리고 싶다', '2'],
 ['마누라의 잔소리에 상처를 입었어. 상처를 고치려면 어떻게 하면 될까?', '0'],
 ['아내와 대화를 나누는 게 잘 안 돼.', '3'],
 ['어제 푸른바다 홍진경보고 빵터짐ㅋ', '5'],
 ['출산해서 아이를 키워야 하는 데 형편이 좋지 않아 걱정이야.', '0'],
 ['새로 산 자동차 앞 좌석 시트에 친한 친구가 커피를 쏟아서 너무 짜증이 나.', '0'],
 ['미래가 없네요 저는..', '3'],
 ['결혼이 코앞인데 내가 옳은 결정을 한 건지 불안해. 식장에 들어가는 게 맞는 건지 감정이 복잡해.', '0'],
 ['아시아 최강국 경제군사 대국 대중화제국 아시아 제3경제대국 선진국 G7대일본제국 제2군사대국 러시아 제국 세계 최강국 미합중국 동북아의 찐따 빵셔틀 개한망국',
  '6'],
 ['요즘 아들과 사이가 안 좋은 것 같아서 괴로워.', '3'],
 ['너무 마음이 상쾌해.', '5'],
 ['유재석 김종민은 연예계성자다', '5'],
 ['서장훈:도대체 이게 무슨 의미가 있습니까~', '4'],
 ['정말 난감하네.', '0'],
 [' 이렇게 논란이 많은 

In [ ]:
#토큰화
#string error 해결 -> vocab 추가
tok=tokenizer.tokenize
data_train = BERTDataset(train, 0, 1, tok,vocab, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
#torch 형식의 dataset을 만들기
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

**5. KoBERT 학습모델 만들기**

In [ ]:
#KoBERT학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

**6. KoBERT모델 학습시키기**

In [ ]:
#KoBERT 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/860 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.01072359085083 train acc 0.15625
epoch 1 batch id 201 loss 1.5303950309753418 train acc 0.26072761194029853
epoch 1 batch id 401 loss 1.1271412372589111 train acc 0.40067799251870323
epoch 1 batch id 601 loss 1.0628498792648315 train acc 0.47602953410981697
epoch 1 batch id 801 loss 1.0867687463760376 train acc 0.5191752496878901
epoch 1 train acc 0.5284595160738714


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/369 [00:00<?, ?it/s]

...

epoch 9 test acc 0.6982554200542005


  0%|          | 0/860 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.007181211840361357 train acc 1.0
epoch 10 batch id 201 loss 0.05725574120879173 train acc 0.9839863184079602
epoch 10 batch id 401 loss 0.17477378249168396 train acc 0.9859336034912718
epoch 10 batch id 601 loss 0.06826642155647278 train acc 0.986662853577371
epoch 10 batch id 801 loss 0.027229446917772293 train acc 0.987144975031211
epoch 10 train acc 0.9872093023255814


  0%|          | 0/369 [00:00<?, ?it/s]

epoch 10 test acc 0.6986506323396566


In [ ]:
#학습시킨 모델 파일로 저장
import joblib
joblib.dump(model, './kobert_trained_model_ver4.pkl')

['./kobert_trained_model_ver4.pkl']

In [ ]:
#print(model)

In [ ]:
#학습시킨 모델 파일 불러오기
#import joblib
#loaded_model = joblib.load('kobert_trained_model_ver4.pkl')